In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import shutil
import os

from tensorflow.keras.utils import image_dataset_from_directory
from keras.applications.mobilenet_v3 import preprocess_input
from keras.applications.mobilenet_v3 import MobileNetV3Large
from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adagrad

import matplotlib.pyplot as plt

### Step 0: Creating the correct directory structure

In this step we need to create as many subfolders as the number of classes, and copy images from ./dataset/no_label folder to ./dataset/images/{class} for each class. 
For instance, for 2 classes we should have:

- ./dataset/images/**0**/img{i}.jpg (for each i belonging to class 0)
- ./dataset/images/**1**/img{i}.jpg (for each i belonging to class 1)

In [ ]:
def create_directories():
    # Loading the dataframe containing a map between the imageId and the corresponding label
    data = pd.read_csv('./dataset/geotags_185K_label.csv').values.tolist()
    
    # Creating the correct directory structure
    for im in data:
        olddir = f'./dataset/no_label/{im[0]}.jpg'
        newdir = f'./dataset/images/{im[1]}/{im[0]}.jpg'
        
        # Creating directory if it does not exist
        os.makedirs(os.path.dirname(newdir), exist_ok=True)
        # Copying the image
        shutil.copyfile(olddir, newdir)

### Step 1: Loading all images with labels

In this step we need to load images specifying the size, the number of batches and the preprocess steps.

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

We are using `MobileNetV3Large` model which already contains a preprocess layer, and thus the `preprocess_input` is just a pass-through function. 
For completeness, and in case we need to change the model we use it, although it can be skipped.

In [ ]:
def preprocess(images, labels):
    return preprocess_input(images), labels

Loading the dataset, and assigning label depending on the directory structure. Also we split the dataset in train and validation, using 20% of images for the validation. 

In [ ]:
train_dataset, validation_dataset = image_dataset_from_directory(
    './dataset/images',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    # interpolation='nearest',
    seed=123,
    validation_split=0.2,
    subset='both')

In [ ]:
class_names = train_dataset.class_names

# Number of classes
NUM_CLASSES = len(class_names)

Plotting 9 images to have a clue of what the dataset contains

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(np.argmax(labels[i]))
        plt.axis("off")

We do not have a test dataset, so we need to create one. To do so, we determine how many batches of data are available in the validation set, then move 20% of them to a test set.

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)

test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Also, we use buffered prefetching to load images from disk without having I/O become blocking.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Step 2 (optional): Data augmentation

We might apply random transformations to the training images, such as rotation and horizontal flipping, to increase the dataset size and to reduce the overfitting.

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

As example, plotting 9 transformations on the same input image

In [ ]:
for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

### Step 3: Defining the model (1)

In the first part, we freeze all layers of the base_model (MobileNetV3) setting the `trainable` parameter to `False`, and we add a Dense layer with as many neurons as the number of classes. Since we want the probability for an image to belong to a certain cell in the Switzerland grid, we need the Softmax as activation function.

Moreover, since we will fine-tune MobileNetV3 later, it is important to set `training` to `False` because that forces to use Batch Normalization layers as inference layers even when they are unfrozen. Otherwise, the updates applied to the non-trainable weights will destroy what the model has learned.

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
inputs = Input(shape=IMG_SHAPE)

base_model = MobileNetV3Large(
    include_top=False,
    weights='imagenet',
    input_tensor=inputs
)
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')

x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

Specifying the optimizer, the learning rate and the loss function to compile the model.

In [ ]:
LEARNING_RATE=0.045

model.compile(optimizer=Adagrad(learning_rate=LEARNING_RATE), loss='categorical_crossentropy')
model.summary()

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

Plotting the learning curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Step 4: Defining the model (2)

In this step we want to fine-tune the MobileNetV3 making all layers trainable, but keeping a learning rate that is 10 times smaller than the previous one.

In [ ]:
base_model.trainable = True

model.compile(optimizer=Adagrad(learning_rate=LEARNING_RATE/10), loss='categorical_crossentropy')
model.summary()

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Plotting the learning curves

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


### Step 5: Evaluating the model

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)